In [1]:
from keras.applications.vgg16 import VGG16
import os
from glob import glob
import tensorflow as tf
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from keras.utils.np_utils import to_categorical # convert to one-hot-encoding

from keras.preprocessing.image import ImageDataGenerator
from keras import layers
from keras import Model
from keras.applications.densenet import DenseNet201
from tensorflow.keras.optimizers import Adam
from keras.callbacks import ReduceLROnPlateau
import keras.backend as K
from keras.models import load_model
%matplotlib inline
import matplotlib.pyplot as plt
import gc

In [2]:
X = np.load("/content/drive/MyDrive/All datasets/skin disease/224_224_X_train.npy")

In [3]:
y=np.load("/content/drive/MyDrive/All datasets/skin disease/224_224_y_train.npy")

In [4]:
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.33333, random_state=42)

In [5]:
X_train.shape, X_val.shape

((5007, 224, 224, 3), (2504, 224, 224, 3))

In [6]:
y_train.shape, y_val.shape

((5007,), (2504,))

In [7]:
y_train = to_categorical(y_train)
y_val = to_categorical(y_val)

In [8]:
class_names = np.array(['akiec', 'bcc', 'bkl', 'df', 'nv', 'vasc', 'mel'])
print(class_names)

['akiec' 'bcc' 'bkl' 'df' 'nv' 'vasc' 'mel']


In [9]:
pre_trained_model = tf.keras.applications.vgg16.VGG16(input_shape=(224, 224, 3), include_top=False, weights="imagenet")

58900480/58889256 [==============================] - 1s 0us/step


In [10]:
pre_trained_model.output

<KerasTensor: shape=(None, 7, 7, 512) dtype=float32 (created by layer 'block5_pool')>

In [11]:
for layer in pre_trained_model.layers:
    print(layer.name)
    layer.trainable = False
    
print(len(pre_trained_model.layers))

input_1
block1_conv1
block1_conv2
block1_pool
block2_conv1
block2_conv2
block2_pool
block3_conv1
block3_conv2
block3_conv3
block3_pool
block4_conv1
block4_conv2
block4_conv3
block4_pool
block5_conv1
block5_conv2
block5_conv3
block5_pool
19


In [12]:
x = layers.GlobalMaxPooling2D()(pre_trained_model.output)
x = layers.Dense(512, activation='relu')(x)
# Add a dropout rate of 0.7
x = layers.Dropout(0.5)(x)
# Add a final sigmoid layer for classification
x = layers.Dense(7, activation='softmax')(x)

In [13]:
model = Model(pre_trained_model.input, x)
optimizer = Adam(lr=0.0001, beta_1=0.9, beta_2=0.999, epsilon=None, decay=0.0, amsgrad=True)
model.compile(loss='categorical_crossentropy',
              optimizer=optimizer,
              metrics=['accuracy'])

/usr/local/lib/python3.7/dist-packages/keras/optimizer_v2/adam.py:105: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(Adam, self).__init__(name, **kwargs)


In [14]:
model.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 224, 224, 3)]     0         
                                                                 
 block1_conv1 (Conv2D)       (None, 224, 224, 64)      1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 224, 224, 64)      36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 112, 112, 64)      0         
                                                                 
 block2_conv1 (Conv2D)       (None, 112, 112, 128)     73856     
                                                                 
 block2_conv2 (Conv2D)       (None, 112, 112, 128)     147584    
                                                                 
 block2_pool (MaxPooling2D)  (None, 56, 56, 128)       0     

In [15]:
train_datagen = ImageDataGenerator(rotation_range=60, width_shift_range=0.2, height_shift_range=0.2,
                                   shear_range=0.2, zoom_range=0.2, fill_mode='nearest')

train_datagen.fit(X_train)

val_datagen = ImageDataGenerator()
val_datagen.fit(X_val)

In [16]:
batch_size = 32
epochs = 3
history = model.fit(train_datagen.flow(X_train,y_train, batch_size=batch_size),
                              epochs = epochs, validation_data = val_datagen.flow(X_val, y_val),
                              verbose = 1, steps_per_epoch=(X_train.shape[0] // batch_size), 
                              validation_steps=(X_val.shape[0] // batch_size))

Epoch 1/3
156/156 [==============================] - 75s 390ms/step - loss: 1.2830 - accuracy: 0.6221 - val_loss: 1.0176 - val_accuracy: 0.6811
Epoch 2/3
156/156 [==============================] - 60s 385ms/step - loss: 1.1041 - accuracy: 0.6639 - val_loss: 0.9723 - val_accuracy: 0.6815
Epoch 3/3
156/156 [==============================] - 60s 385ms/step - loss: 1.0519 - accuracy: 0.6649 - val_loss: 0.9451 - val_accuracy: 0.6907


In [17]:
for layer in model.layers:
    layer.trainable = True

In [18]:
optimizer = Adam(learning_rate=0.0001, beta_1=0.9, beta_2=0.999, epsilon=None, decay=0.0, amsgrad=False)
model.compile(loss='categorical_crossentropy',
              optimizer=optimizer,
              metrics=['acc'])

In [19]:
learning_rate_reduction = ReduceLROnPlateau(monitor='val_acc', patience=3, verbose=1, factor=0.5, 
                                            min_lr=0.000001, cooldown=2)

In [20]:
batch_size = 32
epochs = 40
history = model.fit(train_datagen.flow(X_train,y_train, batch_size=batch_size),
                              epochs = epochs, validation_data = val_datagen.flow(X_val, y_val),
                              verbose = 1, steps_per_epoch=(X_train.shape[0] // batch_size),
                              validation_steps=(X_val.shape[0] // batch_size),
                              callbacks=[learning_rate_reduction])

Epoch 1/40
156/156 [==============================] - 93s 548ms/step - loss: 0.9675 - acc: 0.6708 - val_loss: 0.7802 - val_acc: 0.7175 - lr: 1.0000e-04
Epoch 2/40
156/156 [==============================] - 83s 529ms/step - loss: 0.8388 - acc: 0.6979 - val_loss: 0.7022 - val_acc: 0.7448 - lr: 1.0000e-04
Epoch 3/40
156/156 [==============================] - 83s 529ms/step - loss: 0.7914 - acc: 0.7102 - val_loss: 0.6897 - val_acc: 0.7468 - lr: 1.0000e-04
Epoch 4/40
156/156 [==============================] - 83s 529ms/step - loss: 0.7308 - acc: 0.7401 - val_loss: 0.6555 - val_acc: 0.7572 - lr: 1.0000e-04
Epoch 5/40
156/156 [==============================] - 83s 529ms/step - loss: 0.6915 - acc: 0.7453 - val_loss: 0.6146 - val_acc: 0.7748 - lr: 1.0000e-04
Epoch 6/40
156/156 [==============================] - 83s 528ms/step - loss: 0.6721 - acc: 0.7546 - val_loss: 0.6542 - val_acc: 0.7624 - lr: 1.0000e-04
Epoch 7/40
156/156 [==============================] - 81s 520ms/step - loss: 0.6391 - ac

In [22]:
loss_val, acc_val = model.evaluate(X_val, y_val, verbose=1)
print("Validation: accuracy = %f  ;  loss_v = %f" % (acc_val, loss_val))

79/79 [==============================] - 14s 173ms/step - loss: 0.7314 - acc: 0.8470
Validation: accuracy = 0.847045  ;  loss_v = 0.731366


In [23]:
X_test=np.load("/content/drive/MyDrive/All datasets/skin disease/X_test.npy")
y_test=np.load("/content/drive/MyDrive/All datasets/skin disease/y_test.npy")

In [24]:
X_test.shape, y_test.shape

((2504, 224, 224, 3), (2504,))

In [25]:
y_test = to_categorical(y_test)

In [26]:
loss_test, acc_test = model.evaluate(X_test, y_test, verbose=1)
print("Test: accuracy = %f  ;  loss = %f" % (acc_test, loss_test))

79/79 [==============================] - 12s 151ms/step - loss: 0.8279 - acc: 0.8323
Test: accuracy = 0.832268  ;  loss = 0.827866


In [27]:
model.save("/content/drive/MyDrive/All datasets/skin disease/skin disease with vgg16.h5")